# Хабрахабр

In [6]:
from pymongo import MongoClient

In [25]:
from parsers.text_utils import DefaultTextProcessor, DefaultDocumentProcessor

Перегоняем данные из базы `test.habrahabr` в базу `datasets.habrahabr` с изменением формата и сохраняем на диске в формате Vowpal Wabbit.

In [26]:
client = MongoClient()
in_collection = client["test"]["habrahabr"]
out_collection = client["datasets"]["habrahabr"]

In [27]:
stop_words = open("../datasets/habrahabr/stopwords.txt").read().split()
doc_pipeline = Pipeline([
    ("text-processor",     DefaultTextProcessor()),
    ("document-processor", DefaultDocumentProcessor(stop_lemmas=stop_words)),
])

In [ ]:
%%time

with open("../datasets/habrahabr/habrahabr.txt", "w") as vw_file:
    for doc_id, mongo_doc in enumerate(in_collection.find({ "company_blog": None }), 1):
        doc = {}
        doc["_id"] = "habr_%d" % doc_id
        doc["title"] = mongo_doc["title"]
        doc["url"] = mongo_doc["url"]
        doc["modalities"] = doc_pipeline.fit_transform(mongo_doc["content_html"])
        doc["modalities"]["flat_tag"] = mongo_doc["tags"]
        doc["modalities"]["authors"] = [mongo_doc["author_user"]]
        doc["modalities"]["hubs"] = mongo_doc["hubs"]
        doc["markdown"] = mongo_doc["content_html"]
        # Записать в Vowpal Wabbit
        modalities_str = " ".join(map(lambda p: "|%s %s" % (p[0], " ".join(p[1])), doc["modalities"].items()))
        vw_file.write("%s %s\n" % (doc["_id"], modalities_str))
        # Записать в MongoDB
        out_collection.insert_one(doc)

---